# Example: The Pricing of United States Treasury Coupon Bearing Notes and Bonds
United States Treasury coupon Notes, or Bonds, are a type of fixed-income debt security that investors can buy and sell on the market. When investors purchase a note or bond, they receive periodic coupon payments during the security's lifetime. At the termination of the bond, the investor receives a defined amount, called the face or par value. 

## Learning objectives
In this `example,` students will familiarize themselves with calculating the prices of United States Treasury coupon notes and bonds. 

* __Prerequisite__: Before we begin any calculations, we'll load a United States Treasury action dataset for Treasury notes (`T-notes` or just `notes`) and bonds (`T-bonds` or just `bonds`).
* __Task 1__: First, we'll visualize the cashflows and discounts for a coupon-bearing Treasury bond
    * `Check`: Are the computed and observed bond prices `similar`? 
* __Task 2__: Next, we'll compute the price of a collection of recent `T-notes` and `T-bonds` issued by the United States Treasury and validate our simulated prices using Treasury auction data from `2022` and `2023` for these instruments.
    * `Check`: How well do we estimate the price of notes and bonds at auction?
    * `Discussion question`: Are any of the `notes` or `bonds` mispriced?

## Setup
Set up the computational environment using the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl).

In [3]:
include("Include.jl");

Precompiling VLQuantitativeFinancePackage
  ✓ StatsFuns → StatsFunsInverseFunctionsExt
  ✓ NLSolversBase
  ✓ Distributions → DistributionsTestExt
  ✓ LsqFit
  ✓ LineSearches
  ✓ Optim
  ✓ MathOptInterface
  ✓ Optim → OptimMOIExt
  ✓ MadNLP
  ✓ JuMP
  ✓ VLQuantitativeFinancePackage
  11 dependencies successfully precompiled in 43 seconds. 125 already precompiled.
Precompiling FileIOExt
  ✓ Xorg_libICE_jll
  ✓ mtdev_jll
  ✓ EpollShim_jll
  ✓ Xorg_libXdmcp_jll
  ✓ Xorg_xtrans_jll
  ✓ Xorg_libXau_jll
  ✓ LZO_jll
  ✓ gperf_jll
  ✓ Libmount_jll
  ✓ libevdev_jll
  ✓ Libgpg_error_jll
  ✓ Wayland_protocols_jll
  ✓ Xorg_libpthread_stubs_jll
  ✓ Wayland_jll
  ✓ Dbus_jll
  ✓ eudev_jll
  ✓ Libgcrypt_jll
  ✓ OpenSSL
  ✓ XSLT_jll
  ✓ Xorg_xcb_util_jll
  ✓ Xorg_libX11_jll
  ✓ Xorg_libXinerama_jll
  ✓ Xorg_libXfixes_jll
  ✓ Xorg_libxkbfile_jll
  ✓ Xorg_libXrandr_jll
  ✓ Pango_jll
  ✓ Xorg_libXi_jll
  ✓ Xorg_xkbcomp_jll
  ✓ Xorg_libXcursor_jll
  ✓ Xorg_xkeyboard_config_jll
  ✓ HTTP
  ✓ libdecor_jll
  ✓ 

## Prerequisite: Load the Treasury auction dataset
We'll explore `T-note` and `T-bond` prices from United States Treasury auctions between October 2022 and August 2023 downloaded as a `CSV` file using the [Auction query functionality of TreasuryDirect.gov](https://www.treasurydirect.gov/auctions/auction-query/). We load the `CSV` dataset using the local `MyTreasuryNotesAndBondsDataSet()` function, which returns the auction data as the  `dataset::DataFrame` variable.

In [5]:
dataset = MyTreasuryNotesAndBondsDataSet()

Row,CUSIP,Security Type,Security Term,Auction Date,Issue Date,Maturity Date,Price,High Yield,Interest Rate
,String15,String7,String7,String15,String15,String15,Float64,Float64,Float64
1,91282CHW4,Note,7-Year,08/29/2023,08/31/2023,08/31/2030,99.4773,0.04212,0.04125
2,91282CHV6,Note,2-Year,08/28/2023,08/31/2023,08/31/2025,99.9549,0.05024,0.05
3,91282CHX2,Note,5-Year,08/28/2023,08/31/2023,08/31/2028,99.8889,0.044,0.04375
4,912810TU2,Bond,20-Year,08/23/2023,08/31/2023,08/15/2043,98.3742,0.04499,0.04375
5,912810TT5,Bond,30-Year,08/10/2023,08/15/2023,08/15/2053,98.9127,0.04189,0.04125
6,91282CHT1,Note,10-Year,08/09/2023,08/15/2023,08/15/2033,98.9862,0.03999,0.03875
7,91282CHU8,Note,3-Year,08/08/2023,08/15/2023,08/15/2026,99.936,0.04398,0.04375
8,91282CHR5,Note,7-Year,07/27/2023,07/31/2023,07/31/2030,99.475,0.04087,0.04
9,91282CHQ7,Note,5-Year,07/25/2023,07/31/2023,07/31/2028,99.7988,0.0417,0.04125


Let's store the dimension (number of records) of our treasury auction dataset in the `number_of_bills::Int64` variable using [the `nrow(...)` function exported by the DataFrames.jl package](https://dataframes.juliadata.org/stable/lib/functions/#DataAPI.nrow)

In [7]:
number_of_records = nrow(dataset); # number of records in the dataset

## Task 1: Visualize the cashflows and discounts for coupon-bearing Treasury bonds
Unlike zero-coupon Treasury bills, which have only two cash flow events (you give money to the Treasury, and you receive the face (par) value of the `T-note` or `T-bond` at maturity), coupon-bearing Treasury securities are more complicated because of the periodic coupon payments. Thus, it's helpful to visualize the cash flow events of notes and bonds. 
* We begin by building [an instance of the `DiscreteCompoundingModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.DiscreteCompoundingModel) and store this discount model in the `discount_model::DiscreteCompoundingModel` variable:

In [9]:
discount_model = DiscreteCompoundingModel();

Next, let's build an instance of [the `MyUSTreasuryCouponSecurityModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryCouponSecurityModel) using [the `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple}). In this example, we'll compute the price and cashflow for a `T = 20-yr` bond, with a coupon rate of `c = 1.750%`, a yield (discount rate) `rate = 1.850%`, two coupon payments per year, i.e., $\lambda = 2$ and a face (par) value of $V_{P}$ = `100 USD`. Let's store the result in the `test_bond::MyUSTreasuryCouponSecurityModel` variable. The price value reported on [TreasuryDirect.gov](https://www.treasurydirect.gov/marketable-securities/understanding-pricing/#id-for-more-detailed-formulas-and-useful-tables-264977) for this bond is $V_{B}$ = `98.3369 USD`.
* Similar to zero-coupon `T-bills,` we'll use [the `short-cut` syntax](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#Short-cut-syntax), which relies on the [Julia pipe `|>` operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping) and some syntax sugar to compute the coupon-bearing price, discount factors, and cashflow for the instrument. 

In [11]:
test_bond = build(MyUSTreasuryCouponSecurityModel, (
    T = 20.0, rate = 0.01850, coupon = 0.01750, λ = 2, par = 100.0
)) |> discount_model;

Now that we have populated [the `MyUSTreasuryCouponSecurityModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryCouponSecurityModel) stored in the `test_bond` variable, we can pull data from `test_bond` and construct a table. 
* Let's pull the `price,` `discount,` and `cashflow` fields from the `test_bond` variable and store them in the `nominal_computed_price,` `cashflow,` and `discount` variables

In [13]:
nominal_computed_price = test_bond.price;
cashflow = test_bond.cashflow;
discount = test_bond.discount;
println("This computed bond price = $(nominal_computed_price) USD")

This computed bond price = 98.33464924552273 USD


#### Check: Are the computed and observed bond prices `similar`?
Let's use the [isapprox function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) combined with the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check the `similarity` of the computed, and observed bond price. If the price values are different beyond `rtol = 1e-4`, a `false` result is generated, and an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is thrown.

In [15]:
observed_bond_price = 98.3369;
@assert isapprox(observed_bond_price, nominal_computed_price; rtol = 1e-4)

`Unhide` the code block below to see how we build a table holding the `nominal,` `discounted,` and `cumulative` cash flow for this bond using the [PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl). We'll iterate through each period [using a `for-loop`](https://docs.julialang.org/en/v1/base/base/#for) and populate the `bond_data_table` variable. For each iteration of the loop:
* We access values for the discount and cashflow in period `i` and compute the cumulative cashflow in the `sumvalue` variable
* We then store these data along with the nominal cash flow for each period in the `bond_data_table` array and display the data in the `bond_data_table` variable by calling [the `pretty_table(...)` function exported by the PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl) (with optional values for the `header,` and `tf` arguments). See the [PrettyTables.jl package documentation for information about the `pretty_table(...)` function](https://ronisbr.github.io/PrettyTables.jl/stable/).

In [17]:
number_of_periods = length(cashflow)
bond_data_table = Array{Any,2}(undef, number_of_periods, 5);
sumvalue = 0.0;
for i ∈ 0:(number_of_periods - 1)
    
    discount_value = discount[i]
    payment = cashflow[i];
    sumvalue += payment;

    bond_data_table[i+1,1] = i;
    bond_data_table[i+1,2] = discount_value;
    bond_data_table[i+1,3] = discount_value*payment;
    bond_data_table[i+1,4] = payment;
    bond_data_table[i+1,5] = sumvalue;
end
pretty_table(bond_data_table; 
    header=["Period", "Discount factor", "Nominal cashflow", "Discounted cashflow", "Cumulative cashflow"], tf = tf_simple)

========= ================= ================== ===================== ======================
  Period   Discount factor   Nominal cashflow   Discounted cashflow   Cumulative cashflow 
========= ================= ================== ===================== ======================
       0               1.0           -98.3346              -98.3346              -98.3346
       1           1.00925              0.875               0.86698              -97.4677
       2           1.01859              0.875              0.859034              -96.6086
       3           1.02801              0.875              0.851161              -95.7575
       4           1.03752              0.875               0.84336              -94.9141
       5           1.04711              0.875               0.83563              -94.0785
       6            1.0568              0.875              0.827972              -93.2505
       7           1.06657              0.875              0.820383              -92.4301
     

## Task 2: Compute the price of notes and bonds observed at auction
Next, we compute the price of treasury `notes` and `bonds` from the `dataset::DataFrame` and compare the estimated cost with the price observed at the auction. We process each entry in the `dataset` using [a `for-loop`](https://docs.julialang.org/en/v1/base/base/#for). During each iteration of the loop:
* We get data from the `dataset::DataFrame` and update the model instance. We set the duration `T` field (which we convert to the number of years using the `securityterm` function), the yield, i.e., the $\bar{r}$ value in the discount rate, and coupon rate $c$. We compute the price of the `note (or bond)` on `L8` using the short-cut syntax and the discount model. 
* Finally, we calculate the percentage error between the estimated and observed price and store the data for each iteration in the `computed_price_table::DataFrame` using [the `push!(...)` function](https://dataframes.juliadata.org/stable/lib/functions/#Base.push!). Lastly, we store the populated model instance in the `security_dictionary::Dict{Int64, MyUSTreasuryCouponSecurityModel}` dictionary where keys are the index `i,` values are the populated model instances.

In [19]:
computed_price_table = DataFrame();
security_dictionary = Dict{Int64,MyUSTreasuryCouponSecurityModel}()
for i ∈ 1:number_of_records

    # get data from the dataset 
    T = dataset[i, Symbol("Security Term")] |> String |> securityterm; # what is happening here?
    r̄ = dataset[i, Symbol("High Yield")]
    c = dataset[i, Symbol("Interest Rate")]

    # build the model with the data from this instrument 
    model = build(MyUSTreasuryCouponSecurityModel, (
            par = 100.0, T = T, rate = r̄, coupon = c, λ = 2
        )) |> discount_model;
       
    # compute the percentage error between the computed and observed price
    VB = dataset[i, :Price];
    price_computed = model.price
    error = abs((VB - price_computed)/(price_computed));

    # package up the results -
    results_tuple = (
        CUSIP = dataset[i, :CUSIP],
        term = dataset[i, Symbol("Security Term")],
        rate = r̄*100,
        coupon = c*100,
        computed =  price_computed,
        actual = VB,
        relative_error = error
    );

    # store -
    push!(computed_price_table, results_tuple)
    security_dictionary[i] = model;
end
computed_price_table

Row,CUSIP,term,rate,coupon,computed,actual,relative_error
,String15,String7,Float64,Float64,Float64,Float64,Float64
1,91282CHW4,7-Year,4.212,4.125,99.4773,99.4773,3.59567e-9
2,91282CHV6,2-Year,5.024,5.0,99.9549,99.9549,3.56315e-9
3,91282CHX2,5-Year,4.4,4.375,99.8889,99.8889,3.89252e-9
4,912810TU2,20-Year,4.499,4.375,98.3759,98.3742,1.68543e-5
5,912810TT5,30-Year,4.189,4.125,98.9127,98.9127,2.72566e-9
6,91282CHT1,10-Year,3.999,3.875,98.9862,98.9862,2.16414e-9
7,91282CHU8,3-Year,4.398,4.375,99.936,99.936,7.11434e-10
8,91282CHR5,7-Year,4.087,4.0,99.475,99.475,1.6564e-9
9,91282CHQ7,5-Year,4.17,4.125,99.7988,99.7988,2.74693e-9


### Check: How well do we estimate the price of notes and bonds at auction?
Let's specify a `tolerance` and compute the fraction of notes and bonds with a relative error _less than or equal_ to the `tolerance.`
* Let the `tolerance = 1e-3`. You can specify a different value by setting the `tolerance` variable
* We iterate through the entries of the `computed_price_table,` and increment the `counter` variable if the `relative_error` $\leq$ `tolerance.`
* Finally, we compute the fraction of notes and bonds that satisfy the relative error check

In [21]:
tolerance = 1e-3
counter = 0.0
for i ∈ 1:number_of_records
    if (computed_price_table[i,:relative_error] ≤ tolerance)
        counter += 1
    end
end
fraction = (counter/number_of_records);
println("What fraction of instruments satisfy the error tolerance: $(fraction) \
    or $(counter) out of $(number_of_records)")

What fraction of instruments satisfy the error tolerance: 0.9830508474576272 or 58.0 out of 59


### Discussion question
1. If `fraction` does not equal `1.0`, which note or bond was mispriced? Did the buyer of the mispriced note or bond get a `good` or `bad` deal, i.e., did they underpay or overpay?

In [23]:
### fill in solution here

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.